In [10]:
using JuMP, Gurobi, Test, Combinatorics, LightGraphs, TimerOutputs

# function solveMP()
const gurobi_env = Gurobi.Env()
# setparams!(gurobi_env, OutputFlag = 0)

ins = 2 #ARGS[1] #:10
dataSet = "N50_d20_"
println("Running...")
global prob = "" #"L"
#     include("./TestInstances/"*fileName*".jl")
global q = []
include("./TestInstances/"*dataSet*string(ins)*".jl")
global numArcs = length(arcs[:,1])
global lb = 1
global ub = 999
# println("1")
function setQ(prob)
    global lb, ub, numArcs, q
    if prob == "L"
        lb = 1
        ub = 333
    elseif prob == "M"
        lb = 334
        ub = 666
    elseif prob == "H"
        lb = 667
        ub = 999
    end
    new_q = rand(lb:ub, numArcs)./1000
       
    return new_q
end
# println("2")
if prob != ""
    q = setQ(prob)
end
println("q = ", q)
# q = [0.28, 0.23, 0.033, 0.257, 0.208, 0.037, 0.009, 0.269, 0.302, 0.167, 0.098, 0.288, 0.187]
# println("3")
#     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#     global newP, newM, P_set, M_set, numPaths,numY

include("./functionEnumFeasX.jl")
include("./functionFindQ.jl")
include("./functionGetCost_SP.jl")

include("./functionShortestPathBellmanFord.jl")
include("./functionGenMonitoring.jl")
include("./functionLP_findM.jl")

#     global X_feas = EnumX(arcs, b_x, numArcs, d_x)
#     global EnumY = EnumX(arcs, b_y, numArcs, d_y)
M_start = findall(d_x.<=b_x)
global M_set = [[M_start[1]]]#[EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
global numY = 1 
global R = -sum(log10(1-q_) for q_ in q)  #10^6#0^6
println("R = ", R)
path, gx = shortestPath_BellmanFord(q, arcs, numArcs, origin, destination)
arcsinPath = findall(path.>0)
global P_set = [arcsinPath]

global y_sol = []
#     global 
global numPaths = 1
global newP = true
global newM = true
global iter = 0
global theta_inc = 1
global x_inc = []
global y_inc = []
#     global theta_now = -1e12
#     global inner_Obj = 1-1e12
conRefNum = 20# max(200,2*length(P_set))
con = Array{JuMP.ConstraintRef}(undef, conRefNum)
global k = 0
global rootOptGap = 0
model = direct_model(Gurobi.Optimizer())
set_optimizer_attribute(model, "LazyConstraints", 1)
# set_optimizer_attribute(model, "OutputFlag", 0)
global loc = "./Documents/GitHub/UncertainTarget/EnumX_RowCol2ndStage/TestInstances/"
# set_optimizer_attribute(model, "LogFile", loc*"L_"*dataSet*prob*"_"*string(ins)*".log")

@variable(model, x[1:numArcs], Bin)
@variable(model, theta >= 1-R)#1e6)
@objective(model, Min, theta)
@constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
    
global last_theta = -1e6 - 1
global start = time()
cb_calls = Cint[]
function my_callback_function(cb_data, cb_where::Cint)
    global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y, y_sol
    global newP, newM, P_set, M_set, numPaths,numY, iter, theta_now, inner_Obj
    global theta_inc, x_inc, y_inc
    global k, last_theta
        # You can reference variables outside the function as normal
        push!(cb_calls, cb_where)
        iter += 1
        MIPGAP = 0
        if cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cdouble}()
            GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_NODCNT, resultP)
            node = resultP[]
#             println("\n\tMIPNODE_NODCNT ", node)
            if node == 0
                resultP = Ref{Cdouble}()
                GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_OBJBST, resultP)
                obj = resultP[]
#                 println("\tMIPNODE_OBJBST ", obj)
                resultP = Ref{Cdouble}()
                GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_OBJBND, resultP)
                objBound = resultP[]
#                 println("\tMIPNODE_OBJBND ", objBound)
                MIPGAP = (obj - objBound)/obj*100
#                 println("\tMIPGAP = ", MIPGAP)
                if resultP[] != GRB_OPTIMAL
                    return  # Solution is something other than optimal.
                end
            end
        end
        # You can query a callback attribute using GRBcbget
        if cb_where == GRB_CB_MIPSOL #|| cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cint}()
            # Before querying `callback_value`, you must call:
            ret = GRBcbget(cb_data, cb_where, GRB_CB_MIPSOL_OBJBST, resultP)
            Gurobi.load_callback_variable_primal(cb_data, cb_where)
#         if resultP[] != GRB_OPTIMAL
#             return  # Solution is something other than optimal.
#         end
#         if resultP == GRB_OPTIMAL
#             println("\niter = ", iter)
            x_val = callback_value.(Ref(cb_data), x)
            theta_val = callback_value(cb_data, theta)
#             println("x_val = ", findall(x_val.>0))#, "\t theta_val", theta_val)
            P_set, M_set, lambda, pi_, inner_Obj, y_sol = solveLP_findM(x_val)
#             println("M = ", length(M_set), "\t P = ", length(P_set)) 
#             println("P = ", P_set)
#             println("theta_val = ", theta_val, "\tinner_Obj = ", inner_Obj)
#             println("lambda = ", lambda, "\t pi_ = ", pi_)
            
            numPaths = length(P_set)
            numY = length(M_set)
            lambda_pos = findall(lambda.>0)
            
            TOL = 10^(-3)
#             println("(inner_Obj - theta_val) = ", (inner_Obj - theta_val))
            
#             println("TOL = ", TOL)
            if (inner_Obj - theta_val) > TOL  
                k+=1 
                con_ = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 println("Added con")
#                 println("Type = ", typeof(con_))
#             cut_vio = (sum(lambda[i]*(-R)*sum(x_val[a] for a in P_set[i]) for i=1:numPaths) + pi_) - theta_val
#                 println("Add con : cut violation = ", cut_vio)
#                 println(theta, ">=", sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
                
                w_con = MOI.submit(model, MOI.LazyConstraint(cb_data), con_)

                X_set = []
                for i in lambda_pos
                    X_set = vcat(X_set,P_set[i])
                end
                unique!(X_set)
                c2 = @build_constraint(sum(x[a] for a in X_set) >= 1)
#                 MOI.submit(model, MOI.UserCut(cb_data), c2)
                MOI.submit(model, MOI.LazyConstraint(cb_data), c2)
            end
            if inner_Obj < theta_inc
#                 println(iter, "Updating Incumbent Sol : ", inner_Obj, " \t ", findall(x_val.>0))
                theta_inc = inner_Obj
                x_inc = x_val
                y_inc = y_sol
            end
            
#         if last_theta > theta_val
#             GRBterminate(backend(model))
#         else
#             last_theta = theta_val
#         end
#         end 
    end
        
    return
end
#     You _must_ set this parameter if using lazy constraints.
MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
#     println(model)
optimize!(model)
println("theta_inc ", theta_inc)
println("x_inc ", findall(x_inc.>0))
#     println(model)
totalTime = time() - start
x_sol = JuMP.value.(x)
theta_sol = JuMP.value.(theta)
println("dataSet ", dataSet*string(ins)*prob)
# println("x = ", x_sol)
# println("x_i = ", findall(x_sol.>0))
# println("theta = ", theta_sol)
y_pos = findall(y_inc.>0)
println("M_set = ", length(M_set))
# y_pos = findall(y_sol .> 0)
println("y = ", y_inc[y_pos])
println("M_pos = ", M_set[y_pos])
nodeCount = MOI.get(model, MOI.NodeCount())
println("Node count = ", nodeCount)
println("Total time = ", totalTime)
# timesFile = open(loc*"O_"*dataSet*prob*".txt", "a")
# println(timesFile, dataSet*prob, "; Ins ", ins, "; Time ", totalTime, "; theta ", theta_sol,"; x_sol ", findall(x_sol.>0),"; M_select ", M_set[y_pos], "; y_sol ", y_sol[y_pos], "; Iter ", iter, "; M_len ", length(M_set), "; P_len ", length(P_set), "; NodeCount ", nodeCount)
# close(timesFile)

# if prob != ""
#     qFile = open(loc*"q_"*dataSet*prob*".txt", "a")
#     println(qFile, dataSet*prob, "; Ins ", ins, "; q ", q)
#     close(qFile)
# end

Academic license - for non-commercial use only - expires 2022-12-04
Running...


q = [0.479, 0.621, 0.815, 0.537, 0.709, 0.339, 0.308, 0.314, 0.05, 0.351, 0.071, 0.466, 0.413, 0.549, 0.65, 0.096, 0.027, 0.403, 0.906, 0.728, 0.196, 0.001, 0.745, 0.513, 0.783, 0.481, 0.389, 0.455, 0.37, 0.331, 0.978, 0.968, 0.928, 0.933, 0.861, 0.104, 0.446, 0.852, 0.482, 0.774, 0.373, 0.318, 0.696, 0.916, 0.349, 0.86, 0.717, 0.806, 0.744, 0.218, 0.938, 0.515, 0.851, 0.637, 0.373, 0.324, 0.448, 0.747, 0.839, 0.229, 0.747, 0.667, 0.705, 0.74, 0.238, 0.683, 0.062, 0.154, 0.31, 0.596, 0.145, 0.032, 0.561, 0.631, 0.138, 0.554, 0.763, 0.275, 0.881, 0.338, 0.146, 0.722, 0.46, 0.887, 0.7, 0.251, 0.652, 0.235, 0.744, 0.994, 0.79, 0.856, 0.214, 0.226, 0.198, 0.151, 0.947, 0.27, 0.764, 0.046, 0.69, 0.56, 0.648, 0.957, 0.814, 0.774, 0.538, 0.043, 0.558, 0.333, 0.148, 0.422, 0.333, 0.608, 0.385, 0.496, 0.408, 0.983, 0.277, 0.336, 0.953, 0.928, 0.206, 0.64, 0.621, 0.886, 0.884, 0.94, 0.461, 0.089, 0.027, 0.657, 0.671, 0.593, 0.427, 0.172, 0.568, 0.015, 0.379, 0.508, 0.396, 0.846, 0.536, 0.876, 0.

In [4]:
M_select = Array{Int64,1}[[1, 4, 7, 29], [4, 6], [1, 3], [2, 4, 111], [1, 2, 4, 72], [4, 58, 117], [4, 113, 118], [4, 110, 114], [4, 112, 121], [2, 4, 154], [4, 109, 110], [2, 4, 116], [1, 115], [4, 114, 118, 119]]; 
y_sol = [0.180859, 0.113117, 0.374569, 0.00536602, 0.0584967, 0.00322871, 0.00359874, 0.00631916, 0.00256964, 0.235841, 0.00397533, 0.00461469, 0.00222878, 0.00521762]

1-element Array{Int64,1}:
 3

In [ ]:
findall(x_inc.>0)

In [19]:
a = [1, 2, 3]
b = [4,2,3]

A = [a,b]
# println("A ", A)
# B = union(A[i,:] for i =1:2)
reduce(vcat, A)

6-element Array{Int64,1}:
 1
 2
 3
 4
 2
 3

In [11]:
for i in A
    println(i)
end
union(A[1],A[2])

[1, 2, 3]
[4, 2, 3]


4-element Array{Int64,1}:
 1
 2
 3
 4

In [20]:
a

3-element Array{Int64,1}:
 1
 2
 3

In [25]:
b = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [26]:
issubset(b,a)

true

In [27]:
issubset(a,b)

true